**1. Introduction Project**
# Mini Project: Analisis CCTV Jalan Tol - Vehicle Detection (YOLOv11)
**Nama:** [Febriansyah Syafaat]  
**Bootcamp:** AIML Dibimbing.Id - AI Application Series [cite: 6, 9]  
**Objective:** Membangun model deteksi kendaraan (Car, Truck, Bus) menggunakan YOLOv11[cite: 14, 65, 71].

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**2. Setup Environment**

In [2]:
!pip install ultralytics

import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
!pip install ultralytics

**3. Dataset Preparation**
## 1. Dataset Preparation
Tahap ini meliputi pemisahan data gambar menjadi **train set** dan **val set** sesuai panduan[cite: 63, 67, 68].  
Dataset telah di-anotasi menggunakan **Label Studio** dengan format **YOLO**[cite: 65, 67].

**4. Extract Dataset**

In [4]:
from google.colab import drive
import os

drive.mount('/content/drive')

!mkdir -p /content/dataset/train/images /content/dataset/train/labels
!mkdir -p /content/dataset/val/images /content/dataset/val/labels

!unzip /content/drive/MyDrive/train_images.zip -d /content/dataset/train/images/
!unzip /content/drive/MyDrive/val_images.zip -d /content/dataset/val/images/

print

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/train_images.zip
   creating: /content/dataset/train/images/train_images/
  inflating: /content/dataset/train/images/__MACOSX/._train_images  
  inflating: /content/dataset/train/images/train_images/train_img_29.jpg  
  inflating: /content/dataset/train/images/__MACOSX/train_images/._train_img_29.jpg  
  inflating: /content/dataset/train/images/train_images/train_img_15.jpg  
  inflating: /content/dataset/train/images/__MACOSX/train_images/._train_img_15.jpg  
  inflating: /content/dataset/train/images/train_images/train_img_14.jpg  
  inflating: /content/dataset/train/images/__MACOSX/train_images/._train_img_14.jpg  
  inflating: /content/dataset/train/images/train_images/train_img_28.jpg  
  inflating: /content/dataset/train/images/__MACOSX/train_images/._train_img_28.jpg  
  inflating: /content/dataset/train/images/train_im

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving project-1-at-2026-01-07-11-32-e711ee87.json to project-1-at-2026-01-07-11-32-e711ee87.json


In [6]:
import json
import os
import glob

json_files = glob.glob('/content/*.json')

if not json_files:
    print("Waduh, file JSON tetep gak ketemu bro. Coba upload ulang!")
else:
    json_file = json_files[0]
    print(f"Nemuin file: {json_file}")

    output_dir = '/content/dataset/labels'
    os.makedirs(output_dir, exist_ok=True)

    class_map = {"car": 0, "bus": 1, "truck": 2}

    with open(json_file, 'r') as f:
        data = json.load(f)

    for item in data:
        if 'file_upload' in item:
            raw_name = item['file_upload'].split('-')[-1]
            txt_name = os.path.splitext(raw_name)[0] + ".txt"

            with open(os.path.join(output_dir, txt_name), 'w') as f_out:
                if 'annotations' in item and item['annotations']:
                    for ann in item['annotations'][0].get('result', []):
                        if ann['type'] == 'rectanglelabels':
                            val = ann['value']
                            label_name = val['rectanglelabels'][0]
                            class_id = class_map.get(label_name, 0)

                            x_center = (val['x'] + val['width'] / 2) / 100
                            y_center = (val['y'] + val['height'] / 2) / 100
                            w = val['width'] / 100
                            h = val['height'] / 100

                            f_out.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    print(f"MANTAP! {len(data)} file label udah berhasil dibuat di {output_dir}")

Nemuin file: /content/project-1-at-2026-01-07-11-32-e711ee87.json
MANTAP! 100 file label udah berhasil dibuat di /content/dataset/labels


In [7]:
!ls -lh /content/dataset/labels | head -n 5

total 400K
-rw-r--r-- 1 root root 152 Jan  7 13:36 train_img_0.txt
-rw-r--r-- 1 root root 190 Jan  7 13:36 train_img_10.txt
-rw-r--r-- 1 root root 114 Jan  7 13:36 train_img_11.txt
-rw-r--r-- 1 root root  38 Jan  7 13:36 train_img_12.txt


**5. Model Training**
## 2. Model Training (Step 3)
Melakukan training menggunakan **YOLOv11**. [cite_start]Sesuai instruksi, dilakukan eksperimen pada parameter training[cite: 71, 72].

**6. Training Process**

In [9]:
import os

image_dir = '/content/dataset/images'
label_dir = '/content/dataset/labels'

images = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')}
labels = {os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.txt')}

images_to_delete = images - labels

for img_name in images_to_delete:
    img_path = os.path.join(image_dir, img_name + '.jpg')
    if os.path.exists(img_path):
        os.remove(img_path)

print(f"Pembersihan Selesai!")
print(f"Menghapus {len(images_to_delete)} gambar tanpa label.")
print(f"Sisa Images sekarang: {len(os.listdir(image_dir))}")
print(f"Sisa Labels sekarang: {len(os.listdir(label_dir))}")

Pembersihan Selesai!
Menghapus 100 gambar tanpa label.
Sisa Images sekarang: 100
Sisa Labels sekarang: 100


In [15]:
import os

os.makedirs('/content/dataset/train/images', exist_ok=True)
os.makedirs('/content/dataset/train/labels', exist_ok=True)
os.makedirs('/content/dataset/val/images', exist_ok=True)
os.makedirs('/content/dataset/val/labels', exist_ok=True)

!mv /content/dataset/images/*.jpg /content/dataset/train/images/ 2>/dev/null
!mv /content/dataset/labels/*.txt /content/dataset/train/labels/ 2>/dev/null

!cp /content/dataset/train/images/*[:5] /content/dataset/val/images/ 2>/dev/null
!cp /content/dataset/train/labels/*[:5] /content/dataset/val/labels/ 2>/dev/null

print("✅ Folder siap! Sekarang silakan jalankan cell training lagi.")

✅ Folder siap! Sekarang silakan jalankan cell training lagi.


In [10]:
import yaml

data_config = {
    'train': '/content/dataset/train/images',
    'val': '/content/dataset/val/images',
    'nc': 3,
    'names': ['car', 'truck', 'bus']
}

with open('/content/data.yaml', 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("File data.yaml berhasil diperbarui!")

File data.yaml berhasil diperbarui!


In [16]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

model.train(
    data='/content/data.yaml',
    epochs=50,
    imgsz=640,
    device=0,
    batch=16,
    name='cctv_final_model'
)

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=cctv_final_model5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b1f2c760440>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

**7. Evaluation**
## 3. Evaluation (Step 4)
Melakukan evaluasi menggunakan metrik standar deteksi objek untuk melihat performa model pada validation set[cite: 73].

**8. Metrics Evaluation**

In [18]:
from IPython.display import Image
import os

results_path = '/content/runs/detect/train/results.png'
if os.path.exists(results_path):
    display(Image(filename=results_path, width=800))